In [5]:
# FoML Assign 1 Code Skeleton
# Please use this outline to implement your decision tree. You can add any code around this.

import csv
import numpy as np

# Enter You Name Here
myname = "Ananthoju-Pranav-Sai" # or "Amar-Akbar-Antony"
used_feature = [0,0,0,0,0,0,0,0,0,0,0]  #we use this to keep track of whether the feature is used or not for comparision
# Implement your decision tree below
class DecisionTree():
    tree = {}
    global used_feature  
    def learn(self, training_set):
        # implement this function
        tree={}
        tree['feat_index']=-1               
        tree['threshold']=-1
        tree['subtree']={}
        subtree={}
        X,Y = [row[:-1] for row in training_set],[row[11]for row in training_set]
        n_sam,n_feat = np.shape(X)

        if(n_sam>=2 and self.entropy(Y)!=0):    #stopping conditions

            best_split = self.Best_split(training_set,n_sam,n_feat)
            if(best_split["feat_index"]!=-1):                   
                tree['feat_index']=best_split["feat_index"] 
                tree['threshold']=best_split["threshold"]    
                subtree['left']=DecisionTree.learn(self,best_split["left_dataset"])                     
                subtree['right']=DecisionTree.learn(self,best_split["right_dataset"])  
                subtree['value']=None
                tree['subtree']=subtree   
                self.tree = tree
                # print(self.tree)
                return tree

        leaf_value = max(Y,key=Y.count) 
        subtree['left']=None
        subtree['right']=None
        subtree['value']=leaf_value                                  #if the impurity is zero then we are in the leaf node. 
        tree['subtree']=subtree 
        self.tree=tree
        return tree


    # implement this function
    def classify(self,tree,test_instance):

        if(tree['subtree']['value']!=None):   #if the node is leaf it just returns its value
            return tree['subtree']['value']
        feat_value = test_instance[tree['feat_index']]           
        if feat_value<=tree['threshold']:       #if not then it will check its value with threshold and goes left/right subtree accordingly 
            return self.classify(tree['subtree']['left'],test_instance)
        else:
            return self.classify(tree['subtree']['right'],test_instance)

    def split_dataset(self,dataset,feat_index,threshold):           #splits the dataset according to the feature and its threshold
        left = np.array([row for row in dataset if (row[feat_index])<=threshold])
        right = np.array([row for row in dataset if (row[feat_index])>threshold])
        return left, right

    def Best_split(self,dataset,n_sam,n_feat):  #finds the best split i.e., split with max info gain using all features 
        best_split = {}
        max_info_gain = -np.inf
        #defining the attributes of best_split
        best_split["feat_index"] = -1
        best_split["threshold"] = -1
        best_split["left_dataset"] = []
        best_split["right_dataset"] = []

        for feat_index in range(n_feat):
            feat_values = [row[feat_index] for row in dataset]
            d = float(max(feat_values)-min(feat_values))/25
            possible_thresholds = np.linspace(float(min(feat_values))+d,float(max(feat_values))-d,10) 
            for t in possible_thresholds:
                left_dataset,right_dataset = self.split_dataset(dataset,feat_index,t)
                if(len(left_dataset)>0 and len(right_dataset)>0):
                    y,y_left,y_right = [row[11]for row in dataset],[row[11]for row in left_dataset],[row[11]for row in left_dataset]
                    info_gain_t = self.info_gain(y,y_left,y_right,"entropy")
                    if(info_gain_t>max_info_gain):
                        best_split["feat_index"] = feat_index
                        best_split["threshold"] = t
                        best_split["left_dataset"] = left_dataset
                        best_split["right_dataset"] = right_dataset
                        max_info_gain=info_gain_t
        return best_split

    def info_gain(self,parent,child_1,child_2,way):  #function used to calculate info_gain
        w1 = len(child_1)/len(parent)
        w2 = len(child_2)/len(parent)
        
        if(way=="entropy"):
            gain = self.entropy(parent)-(w1*self.entropy(child_1)+w2*self.entropy(child_2))
        if(way=="gini_index"):
            gain = self.gini_index(parent)-(w1*self.gini_index(child_1)+w2*self.gini_index(child_2))
        return gain

    def entropy(self,label):        #function to calculate impurity using entropy
        label = np.array(label,dtype=int)
        classes = np.unique(label)
        entropy = 0
        for i in classes:
            p_c = len(label[label == i]) / len(label)
            entropy += -p_c * np.log2(p_c)
        return entropy
    
    def gini_index(self,label):     #function to calculate impurity using gini index
        label = np.array(label,dtype=int)
        classes = np.unique(label)
        s=0
        for i in classes:
            p_c = len(label[label == i]) / len(label)
            s += p_c*p_c
        gini = 1-s
        return gini
    
    def print_tree(self):       #function to print the tree
        print(self.tree['feat_index'],self.tree['threshold'])
        if(self.tree['value']!=-1):
            print(self.tree['value'])
        else:
            self.tree['left'].print_tree()
            self.tree['right'].print_tree()

In [6]:
def run_decision_tree():

    # Load data set
    with open("wine-dataset.csv") as f:
        next(f, None)
        data = [tuple(line) for line in csv.reader(f, delimiter=",")]
        data = np.array(data,dtype=float)
    print("Number of records: %d" % len(data))

    # Split training/test sets
    # You need to modify the following code for cross validation.
    accuracy=[]
    for j in range(10):
        K = 10
        training_set = [x for i, x in enumerate(data) if i % K != j]
        test_set = [x for i, x in enumerate(data) if i % K == j]
        
        Tree = DecisionTree()
        # Construct a tree using training set
        tree = Tree.learn( training_set )
        # Classify the test set using the tree we just constructed
        results = []
        for instance in test_set:
            result = Tree.classify(tree,instance[:-1] )
            results.append( result == instance[-1])
                # print(result==instance[-1],result)

            # Accuracy
        accuracy.append(float(results.count(True))/float(len(results)))
        print("Accuracy in ",j+1," fold : ",accuracy[j])
    accuracy = float(sum(accuracy)/len(accuracy))
    print ("accuracy: %.4f" % accuracy)    

    # Writing results to a file (DO NOT CHANGE)
    f = open(myname+"result.txt", "w")
    f.write("accuracy: %.4f" % accuracy)
    f.close()


if __name__ == "__main__":
    run_decision_tree()

Number of records: 4898
Accuracy in  1  fold :  0.8285714285714286
Accuracy in  2  fold :  0.8224489795918367
Accuracy in  3  fold :  0.8081632653061225
Accuracy in  4  fold :  0.8367346938775511
Accuracy in  5  fold :  0.8489795918367347
Accuracy in  6  fold :  0.8367346938775511
Accuracy in  7  fold :  0.8387755102040816
Accuracy in  8  fold :  0.8204081632653061
Accuracy in  9  fold :  0.8323108384458078
Accuracy in  10  fold :  0.8139059304703476
accuracy: 0.8287
